In [2]:
import numpy as np

In [208]:
def gen_pc_data(N, M, R, percent_moonlight):
    
    # First generate W
    W = np.zeros((N,R))
    moonlight_genes = np.random.uniform(0,1,N)
    moonlight_genes = moonlight_genes<percent_moonlight
    
    # Pick the modules for each gene
    for gene_idx in range(N):
        if moonlight_genes[gene_idx]:
            num_mods = np.random.randint(2,R+1)
            gene_mods = np.random.randint(0,R,num_mods)
            W[gene_idx, gene_mods] = 1
        else: 
            gene_mod = np.random.randint(0,R)
            W[gene_idx, gene_mod] = 1
    
    # Pick the relative expression levels of each gene for each module
    for mod_idx in range(R):
        # Handel the case where a module has no genes (very unlikely for N >> R)
        if W[:,mod_idx].sum() == 0:
            W[np.random.randint(0,N),mod_idx] = 1
            
        else:
            col_arr = W[:,mod_idx]
            col_sum = int(col_arr.sum())
            dir_params = np.ones(col_sum)
            gene_expressions_arr = np.random.dirichlet(dir_params).reshape(col_sum,1)
            idx_arr = np.argwhere(col_arr != 0)
            col_arr[idx_arr] = gene_expressions_arr
            W[:, mod_idx] = col_arr
            
    # Then generate H
    H = np.ones((R,M))
    H = np.apply_along_axis(np.random.dirichlet, 1, H)
    
    # Now generate V from W and H
    V = 1e6*np.matmul(W, H)
    V = np.random.poisson(V)
            
    return W, H, V

In [209]:
test = gen_pc_data(10,5,5,0.1)
test

(array([[0.30990095, 0.        , 0.        , 0.35449735, 0.        ],
        [0.        , 0.        , 0.29671097, 0.        , 0.        ],
        [0.26490142, 0.53934698, 0.28076932, 0.22780652, 0.        ],
        [0.34083284, 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.22326401, 0.        ],
        [0.        , 0.46065302, 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.19443212, 0.        ],
        [0.08436479, 0.        , 0.        , 0.        , 0.        ],
        [0.        , 0.        , 0.42251971, 0.        , 0.        ],
        [0.        , 0.        , 0.        , 0.        , 1.        ]]),
 array([[0.15851887, 0.25128794, 0.38471029, 0.10091687, 0.10456603],
        [0.17921383, 0.00777472, 0.19886121, 0.11068229, 0.50346794],
        [0.0200958 , 0.41764935, 0.15896622, 0.18676747, 0.21652115],
        [0.43966004, 0.04590384, 0.31088132, 0.13270936, 0.07084543],
        [0.1986396

In [102]:
else:
            col_arr = W[:,mod_idx]
            dir_params = np.ones(int(col_arr.sum()))
            gene_expressions_arr = np.random.dirichlet(dir_params)
            idx_arr = np.argwhere(W[:, mod_idx] != 0)
            for i in range(len(gene_expressions_arr)):
                col_arr[idx_arr[i]] = gene_expressions_arr[i]
            print(col_arr)
            W[:, mod_idx] = col_arr
            

array([0, 1, 0, 0, 0])